In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, FileLink
from ipywidgets import FileUpload, Output, Button
from io import BytesIO
import os
 
# Set up file upload widget
upload = FileUpload(accept='.json', multiple=False)
display(upload)
 
# Output widgets to display chart and table
output_plot = Output()
output_table = Output()
 
# Button to download CSV
download_button = Button(description="Download Summary CSV")
download_button.layout.visibility = 'hidden'  # Initially hidden
 
# Function to handle file upload
def handle_file_upload(change):
    output_plot.clear_output()
    output_table.clear_output()
   
    if upload.value:
        # Load and process the uploaded JSON data
        file_content = upload.value[list(upload.value.keys())[0]]['content']
        json_data = json.loads(file_content)
       
        # Extract relevant data for each request
        results = json_data.get("results", [])
        data = []
        for result in results:
            # Extract 'name' (e.g., "search-imvs") and test pass/fail details
            name = result["name"]
            tests = result["tests"]
            for test, status in tests.items():
                data.append({
                    "name": name,
                    "test_name": test,
                    "result": "Pass" if status else "Fail"
                })
       
        # Convert to DataFrame
        df = pd.DataFrame(data)
       
        # Generate summary for overall pass/fail counts
        overall_summary = df["result"].value_counts().reindex(["Pass", "Fail", "Skipped"], fill_value=0)
 
        # Display overall summary as a pie chart
        with output_plot:
            plt.figure(figsize=(6, 6))
           
            # Filter out results with 0.0% to avoid displaying 0% categories
            overall_summary = overall_summary[overall_summary > 0]
            overall_summary.plot(kind="pie", autopct="%1.1f%%", colors=["#006400", "#8B0000", "#d3d3d3"])
 
            plt.title("Overall Test Results (Pass/Fail/Skipped)")
            plt.ylabel("")  # Hide y-axis label for cleaner look
            plt.show()
       
        # Detailed pass/fail counts for each request
        per_request_summary = df.groupby(['name', 'result']).size().unstack(fill_value=0)
       
        # Display the table for detailed breakdown
        with output_table:
            display(per_request_summary)
       
        # Save CSV to a file and show download button
        csv_file_path = "/tmp/test_summary.csv"
        per_request_summary.to_csv(csv_file_path)
       
        # Create a downloadable link for the saved CSV file
        download_button.layout.visibility = 'visible'
        download_button.on_click(lambda b: display(FileLink(csv_file_path, result_html_prefix="Click here to download: ")))
 
# Set up observer for file upload
upload.observe(handle_file_upload, names='value')
 
# Display output areas for chart, table, and download button
display(output_plot, output_table, download_button)